In [1]:
from transformers import pipeline

classifier = pipeline("text-classification", model="facebook/bart-large-mnli")

Device set to use cpu


In [5]:
# premise = "The economy is growing rapidly."
# hypothesis = "The economy is in a recession."

premise = "The economy is growing rapidly."
hypothesis = "The government announced new infrastructure projects."

res = classifier([f"{premise} </s> {hypothesis}", f"{premise} </s> {hypothesis}"], top_k=1)
print(res)

[[{'label': 'neutral', 'score': 0.9987346529960632}], [{'label': 'neutral', 'score': 0.9987346529960632}]]


In [1]:
import torch

torch.cuda.is_available()

False

In [2]:
# print torch version and cuda version
print(torch.__version__)
print(torch.version.cuda)

2.8.0+cpu
None


In [12]:
from litellm import completion


tweet1 = "Tesla just announced record profits this quarter. EV demand is booming!"
tweet2 = "Tesla is struggling with massive losses and plummeting EV sales this quarter."

# tweet1 = "Tesla just announced record profits this quarter. EV demand is booming!"
# tweet2 = "Tesla profits this quarter are very high!"

is_comparable_message = (
f"""Given two tweets, determine whether they are discussing the same event, claim, or specific topic in a way that makes them worth comparing for logical contradiction.

Respond only with "Yes" if the tweets are semantically or factually related enough to potentially contradict each other. Respond "No" if they discuss unrelated topics, different products, different times, or do not make comparable claims.

In the reason field also provide a reason for your answer, explaining why the tweets are or are not comparable.

Tweet 1: {tweet1}
Tweet 2: {tweet2}

"Does Tweet 2 contradict Tweet 1?"""
)

message = (
f"""You are given two tweets.

Determine the logical relationship between them by choosing one of the following answers:
- "Contradiction" → if the tweets make opposing or incompatible claims about the same topic or event.
- "No Contradiction" → if the tweets are about the same topic or event, but do not contradict each other.
- "Not Comparable" → if the tweets are about different topics, unrelated events, or lack comparable claims.

Then, provide a short explanation for your answer in the reason field.

Tweet 1: {tweet1}
Tweet 2: {tweet2}

Answer:"""
)

print(message)

response = completion(
            model="ollama/llama3.2:3b-instruct-fp16",
            messages = [{
                "content": message,
                "role": "user"
            }],
            temperature=0,
            api_base="http://localhost:11434",
            response_format={
                "type": "json_schema",
                "json_schema": {
                    "schema": {
                        "type": "object",
                        "properties": {
                            "answer": {"type": "string"},
                            "reason": {"type": "string"},
                        }
                    }
                }
            },
)

response_text = response["choices"][0]["message"]["content"]
response_text = response_text.strip().lower()
print(response_text)

You are given two tweets.

Determine the logical relationship between them by choosing one of the following answers:
- "Contradiction" → if the tweets make opposing or incompatible claims about the same topic or event.
- "No Contradiction" → if the tweets are about the same topic or event, but do not contradict each other.
- "Not Comparable" → if the tweets are about different topics, unrelated events, or lack comparable claims.

Then, provide a short explanation for your answer in the reason field.

Tweet 1: Tesla just announced record profits this quarter. EV demand is booming!
Tweet 2: Tesla is struggling with massive losses and plummeting EV sales this quarter.

Answer:
{"answer": "contradiction", "reason": "the two tweets present opposing claims about the same topic, tesla's financial performance, specifically mentioning record profits in one tweet and massive losses and plummeting ev sales in another."}


In [15]:
import json

json.loads(response_text)

{'answer': 'contradiction',
 'reason': "the two tweets present opposing claims about the same topic, tesla's financial performance, specifically mentioning record profits in one tweet and massive losses and plummeting ev sales in another."}